In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append("../src/utils")
from utils import SimpleGraphVoltDatasetLoader, read_and_prepare_data
from torch_geometric_temporal.signal import temporal_signal_split
import torch

In [2]:
torch.cuda.empty_cache() 

In [3]:
trafo_id = "T1330"

loader = SimpleGraphVoltDatasetLoader(trafo_id)
loader_data = loader.get_dataset(num_timesteps_in=12, num_timesteps_out=4)

train_dataset, test_eval_dataset = temporal_signal_split(loader_data, train_ratio=0.7)
test_dataset, eval_dataset = temporal_signal_split(test_eval_dataset, train_ratio=0.5)


(2, 114)
(114, 5)
(114,)
(70065, 113, 21, 12)
(70065, 113, 4)


In [4]:
train_dataset[0]

Data(x=[113, 21, 12], edge_index=[2, 114], edge_attr=[114], y=[113, 4])

In [15]:
train_dataset[0]

Data(x=[113, 21, 12], edge_index=[2, 114], edge_attr=[114], y=[113, 4])

In [5]:
train_dataset[0].x.shape[1]
train_dataset[0].y.shape[1]

4

In [6]:
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import A3TGCN
from tqdm import tqdm

In [7]:
class TemporalGNN(torch.nn.Module):
    def __init__(self, node_features, periods):
        super(TemporalGNN, self).__init__()
        # Attention Temporal Graph Convolutional Cell
        out_channels = 32
        self.tgnn = A3TGCN(in_channels=node_features, 
                           out_channels=out_channels, 
                           periods=periods)
        # Equals single-shot prediction
        self.linear = torch.nn.Linear(out_channels, periods)

    def forward(self, x, edge_index):
        """
        x = Node features for T time steps
        edge_index = Graph edge indices
        """
        h = self.tgnn(x, edge_index)
        h = F.relu(h)
        h = self.linear(h)
        return h

In [8]:
device = torch.device('cuda') # cuda
subset = 2000

In [12]:
model = TemporalGNN(node_features=train_dataset[0].x.shape[1], periods=train_dataset[0].y.shape[1]).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.MSELoss()
device = torch.device('cuda')
losses = train(model, device, loader_data, optimizer, loss_fn, epochs=2)

0it [00:00, ?it/s]

70065it [05:15, 222.07it/s]
70065it [05:14, 222.94it/s]


In [11]:
def train(model, device, loader_data, optimizer, loss_fn, epochs):
    """
    Definition of the training loop.
    """
    epoch_losses = []
    
    for epoch in range(epochs):
        epoch_loss = 0
        for snapshot in tqdm(loader_data):
            snapshot.to(device)
            optimizer.zero_grad()
            out = model(snapshot.x, snapshot.edge_index)
            loss = loss_fn(out, snapshot.y)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.detach().cpu().numpy()
        epoch_losses.append(epoch_loss)
        
    return epoch_losses
            
    
    

In [ ]:
model = TemporalGNN(node_features=train_dataset[0].x.shape[1], periods=train_dataset[0].y.shape[1]).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.MSELoss()
device = torch.device('cuda')
losses = train(model, device, loader_data, optimizer, loss_fn, epochs=2)

In [13]:
losses

[1547.6376787442714, 912.4260821295902]

In [17]:
#import sleep
import time